In [1]:
import json
from keras.models import model_from_json
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input
from keras.models import model_from_json
from keras.optimizers import SGD, RMSprop, Adagrad

Using TensorFlow backend.


In [2]:
with open('/home/rahul/ICIAR/model_28.json') as json_file:
              model_json = json_file.read()
model = model_from_json(model_json)

In [3]:
model.load_weights('/home/rahul/ICIAR/trained_weights_28.model')

In [4]:
model.layers.pop()

In [5]:
new_model = model.output

In [6]:
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
FC_SIZE = 4096
dropout = 0.5

In [7]:
x1 = Dense(FC_SIZE, activation='relu', name="fc_dense1")(new_model)
x1 = Dropout(dropout, name = 'dropout1')(x1)
x1 = BatchNormalization(name="fc_batch_norm1")(x1)
x1 = Dense(FC_SIZE, activation='relu', name="fc_dense2")(x1)
x1 = Dropout(dropout, name = 'dropout2')(x1)

In [8]:
x2 = Dense(FC_SIZE, activation='relu', name="fc_dense3")(new_model)
x2 = Dropout(dropout, name = 'dropout3')(x2)
x2 = BatchNormalization(name="fc_batch_norm2")(x2)
x2 = Dense(FC_SIZE, activation='relu', name="fc_dense4")(x2)
x2 = Dropout(dropout, name = 'dropout4')(x2)

In [9]:
from keras.layers.merge import concatenate

In [10]:
x12 = concatenate([x1, x2], name = 'mixed11')
x12 = Dropout(dropout, name = 'dropout5')(x12)
x12 = Dense(FC_SIZE//16, activation='relu', name = 'fc_dense5')(x12)
x12 = Dropout(dropout, name = 'dropout6')(x12)
x12 = BatchNormalization(name="fc_batch_norm3")(x12)
x12 = Dense(FC_SIZE//32, activation='relu', name = 'fc_dense6')(x12)
x12 = Dropout(dropout, name = 'dropout7')(x12)

In [11]:
model.layers.pop()

In [12]:
model.layers.pop()

In [13]:
model.layers.pop()

In [14]:
model.layers.pop()

In [15]:
model.layers.pop()

In [16]:
model.layers[-1].outbound_nodes = []
model.outputs = [model.layers[-1].output]
x3 = model.get_layer('mixed10').output
x3 = GlobalAveragePooling2D( name = 'global_avg_pooling2')(x3)
x3 = Dense(2048, activation='relu', name = 'fc_dense7')(x3)
x3 = Dropout(dropout, name = 'dropout8')(x3)
x3 = BatchNormalization(name="fc_batch_norm4")(x3)
x3 = Dense(2048, activation='relu', name = 'fc_dense8')(x3)
x3 = Dropout(dropout, name = 'dropout9')(x3)

In [17]:
xout = concatenate([x12, x3], name ='mixed12')
xout = Dense(FC_SIZE//32, activation='relu', name = 'fc_dense9')(xout)
xout = Dropout(dropout, name = 'dropout10')(xout)

In [18]:
nb_classes =4

In [19]:
predictions = Dense(nb_classes, activation='softmax', name='predictions')(xout)

In [20]:
from keras.models import Model

In [21]:
model_out = Model(inputs=model.input, outputs=predictions)

In [22]:
from keras.optimizers import SGD, RMSprop, Adagrad
sgd = SGD(lr=1e-7, decay=0.5, momentum=1, nesterov=True)
rms = RMSprop(lr=1e-7, rho=0.9, epsilon=1e-08, decay=0.0)
ada = Adagrad(lr=1e-3, epsilon=1e-08, decay=0.0)
optimizer = ada

In [23]:
model_out.compile(optimizer=optimizer, loss='categorical_crossentropy', 
                metrics=['accuracy'])

In [24]:
import os

In [29]:
def save_model(dir_name, name, model):
    file_loc = dir_name
    file_pointer = os.path.join(file_loc+"//trained")
    model.save_weights(os.path.join(file_pointer + "_weights_"+str(name)+".model"))
    
    model_json = model.to_json()                                                # Serialize model to JSON
    with open(os.path.join(file_pointer+"_config_"+str(name)+".json"), "w") as json_file:
        json_file.write(model_json)
    print ("Saved the trained model weights to: " + 
           str(os.path.join(file_pointer + "_weights_"+str(name)+".model")))
    print ("Saved the trained model configuration as a json file to: " + 
           str(os.path.join(file_pointer+"_config_"+str(name)+".json")))

In [30]:
save_model('/home/rahul/', 'model_32', model_out)

Saved the trained model weights to: /home/rahul///trained_weights_model_32.model
Saved the trained model configuration as a json file to: /home/rahul///trained_config_model_32.json
